<a href="https://colab.research.google.com/github/basselkassem/classifier-adversary/blob/master/classifier_adversary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is inspired by the paper [Decorrelated Jet Substructure Tagging using Adversarial Neural Networks](https://arxiv.org/abs/1703.03507).

We illustrate how to use the Adversarial Neural Network to decouple a classifier response from the values of its input features.
In other words, we are interested in making the output of a classifier not dependent on the value distribution of the input features. 
This property is useful to reduce the uncertainty about the effect of the input's features on the classifier's output.
Although this approach reduces the discriminative power of the classifier, but it keeps a relatively good performance.


# Import Libs

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.losses import CategoricalCrossEntropy

print(tf.__version__)

# Load Data

## Data Preprocessing

# Classifier Definition

In [ ]:
class Classifier(Model):
  def __init__(self, ):
    super(Model, self).__init__()

  def __call__(self, ):
    pass


# Adversary Definition

In [ ]:
class Adversary(Model):
  def __init__(self, ):
    super(Model, self).__init__()
    
  def __call__(self, ):
    pass

# Classifier-Adversary Definition

# Optimizers

# Loss Functions

# Training

# Evaluation